In [1]:
import pandas as pd

# Load the data

In [2]:
df_numeric = pd.read_csv("dataset/eeg_with_numeric_labels.csv")
df_text = pd.read_csv("dataset/eeg_with_text_labels.csv")
df_score_vs_distance = pd.read_excel("dataset/score_vs_distance.xlsx")

# Rename the column "eeg_251" to "score"

In [3]:
df_numeric["eeg_251"]
df_numeric_with_score = df_numeric.rename(columns={"eeg_251": "score"})

In [4]:
df_numeric_with_score.head()

,eeg_1,eeg_2,eeg_3,eeg_4,eeg_5,eeg_6,eeg_7,eeg_8,eeg_9,eeg_10,...,eeg_242,eeg_243,eeg_244,eeg_245,eeg_246,eeg_247,eeg_248,eeg_249,eeg_250,score
0,0.7177,-0.2062,0.0475,1.0721,2.4135,3.5125,3.7577,2.7633,0.7151,-1.5270,...,9.8178,9.8659,6.0245,-0.3442,-6.7022,-10.5920,-10.7050,-7.2834,-1.8025,9
1,-20.2720,-15.5270,-7.3470,1.1873,7.7569,11.8050,14.1470,15.7500,16.7530,16.4740,...,-14.8970,-14.8110,-11.2060,-4.8725,1.8731,6.2072,6.1948,1.8273,-4.8429,7
2,-2.5855,-3.6533,-3.1125,-1.5704,-0.2837,-0.3124,-1.8101,-3.9264,-5.3829,-5.3230,...,1.4155,-0.1838,-3.6101,-8.3515,-12.8430,-15.1760,-14.1210,-9.8510,-3.9632,8
3,3.1062,3.9020,4.1897,4.0014,3.4469,2.5849,1.4279,0.0445,-1.3564,-2.4403,...,12.2290,14.7390,16.0430,16.1770,15.4330,14.1300,12.4790,10.6100,8.6729,8
4,4.8654,4.2604,3.1231,1.7460,0.6860,0.5555,1.7299,4.1115,7.0969,9.7958,...,-0.2740,0.2532,0.0332,-1.3112,-3.5680,-5.8331,-6.8665,-5.7612,-2.5252,9


## imbalanced dataset problem occurs!

In [5]:
score_by_count = df_numeric_with_score.groupby("score").size().reset_index(name="counts").sort_values(by="score",ascending=False)
score_by_count

,score,counts
5,10,182
4,9,271
3,8,92
2,7,17
1,6,5
0,0,6


In [37]:
df_score_vs_distance[df_score_vs_distance["score"] == 10].describe()
# print the max min and mean values for each score
score_list = df_score_vs_distance.score.unique()
reversed = np.sort(score_list)[::-1]
reversed
for i in reversed:
    print("Score: ", i)
    print(df_score_vs_distance[df_score_vs_distance["score"] == i].describe())
    print("\n\n")

Score:  10
       score    distance
count  182.0  182.000000
mean    10.0    1.403846
std      0.0    0.754022
min     10.0    0.100000
25%     10.0    0.925000
50%     10.0    1.400000
75%     10.0    1.900000
max     10.0    6.700000



Score:  9
       score    distance
count  271.0  271.000000
mean     9.0    2.874539
std      0.0    0.715838
min      9.0    1.000000
25%      9.0    2.400000
50%      9.0    2.900000
75%      9.0    3.500000
max      9.0    4.700000



Score:  8
       score   distance
count   92.0  92.000000
mean     8.0   4.979348
std      0.0   0.587514
min      8.0   3.700000
25%      8.0   4.500000
50%      8.0   5.000000
75%      8.0   5.500000
max      8.0   6.100000



Score:  7
       score   distance
count   17.0  17.000000
mean     7.0   6.405882
std      0.0   1.569025
min      7.0   0.700000
25%      7.0   6.200000
50%      7.0   6.500000
75%      7.0   7.100000
max      7.0   8.000000



Score:  6
       score  distance
count    5.0  5.000000
mean     

# Fixing the distances of the shootings
the values are not representing the actual distances to center but distance to closest circle. so, I added 12.2 cm * (10-score) to every shooting
for example if score is 10 we wont add additional distance(because 10-10 equals to 0) but if score is 9: we added 12.2*(10-9) cm to distance in recordings

I am not sure if this method is true?



In [10]:
df_score_vs_distance['better_distance'] = df_score_vs_distance['distance'] + (10 - df_score_vs_distance['score']) * 12.2
df_score_vs_distance[df_numeric_with_score["score"] == 8]

,score,distance,actual_distance,better_distance,normalized_form
2,8,5.1,29.5,29.5,0.809585
3,8,4.5,28.9,28.9,0.813472
11,8,4.7,29.1,29.1,0.812176
12,8,4.4,28.8,28.8,0.814119
13,8,5.3,29.7,29.7,0.808290
...,...,...,...,...,...
428,8,4.1,28.5,28.5,0.816062
444,8,5.7,30.1,30.1,0.805699
454,8,4.7,29.1,29.1,0.812176
464,8,5.2,29.6,29.6,0.808938


# Min-max scaling to corrected form

In [16]:
# Min-Max Normalization with inverted scale
min_value = df_score_vs_distance['better_distance'].min()
max_value = df_score_vs_distance['better_distance'].max()
df_score_vs_distance['normalized_form'] = 1 - ((df_score_vs_distance['better_distance'] - min_value) / (max_value - min_value))
df_score_vs_distance[df_numeric_with_score["score"]== 9].describe()

,score,distance,actual_distance,better_distance,normalized_form
count,271.0,271.000000,271.000000,271.000000,271.000000
mean,9.0,2.874539,15.074539,15.074539,0.903015
std,0.0,0.715838,0.715838,0.715838,0.004636
min,9.0,1.000000,13.200000,13.200000,0.891192
25%,9.0,2.400000,14.600000,14.600000,0.898964
50%,9.0,2.900000,15.100000,15.100000,0.902850
75%,9.0,3.500000,15.700000,15.700000,0.906088
max,9.0,4.700000,16.900000,16.900000,0.915155


# Merge the min-max scaled form to data

In [17]:
df_merged = df_numeric_with_score.merge(df_score_vs_distance[["normalized_form"]], left_index=True, right_index=True, how='inner')
df_merged

,eeg_1,eeg_2,eeg_3,eeg_4,eeg_5,eeg_6,eeg_7,eeg_8,eeg_9,eeg_10,...,eeg_243,eeg_244,eeg_245,eeg_246,eeg_247,eeg_248,eeg_249,eeg_250,score,normalized_form
0,0.7177,-0.2062,0.0475,1.0721,2.4135,3.5125,3.7577,2.7633,0.7151,-1.5270,...,9.8659,6.0245,-0.3442,-6.7022,-10.5920,-10.7050,-7.2834,-1.8025,9,0.905440
1,-20.2720,-15.5270,-7.3470,1.1873,7.7569,11.8050,14.1470,15.7500,16.7530,16.4740,...,-14.8110,-11.2060,-4.8725,1.8731,6.2072,6.1948,1.8273,-4.8429,7,0.723446
2,-2.5855,-3.6533,-3.1125,-1.5704,-0.2837,-0.3124,-1.8101,-3.9264,-5.3829,-5.3230,...,-0.1838,-3.6101,-8.3515,-12.8430,-15.1760,-14.1210,-9.8510,-3.9632,8,0.809585
3,3.1062,3.9020,4.1897,4.0014,3.4469,2.5849,1.4279,0.0445,-1.3564,-2.4403,...,14.7390,16.0430,16.1770,15.4330,14.1300,12.4790,10.6100,8.6729,8,0.813472
4,4.8654,4.2604,3.1231,1.7460,0.6860,0.5555,1.7299,4.1115,7.0969,9.7958,...,0.2532,0.0332,-1.3112,-3.5680,-5.8331,-6.8665,-5.7612,-2.5252,9,0.898316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,3.9657,5.3286,5.9334,5.4893,4.1019,2.3230,0.9462,0.6367,1.5860,3.3815,...,0.1096,-2.8541,-5.3169,-6.8093,-7.1329,-6.4084,-5.0492,-3.6339,10,0.986399
569,-0.7658,0.6194,2.6931,4.2242,4.2618,2.7016,0.3394,-1.6152,-2.2325,-1.3196,...,-2.5776,-1.2234,0.1361,0.8815,0.7860,0.1151,-0.5665,-0.7658,10,0.988990
570,7.9655,11.3100,12.2670,10.9120,8.0865,4.9417,2.4252,0.9726,0.4893,0.5447,...,-5.3580,-4.1075,-2.6476,-1.2086,0.0964,1.2556,2.2963,3.1921,10,0.987047
571,-0.6448,1.3528,1.6686,0.5742,-0.8300,-1.3065,-0.2264,2.0804,4.5637,6.1062,...,-4.0059,-3.2825,-2.2467,-0.9428,0.3840,1.3054,1.3794,0.3913,9,0.901554


# Save data for further works

In [19]:
df_merged.to_csv("normalized_data.csv", index=False)